In [3]:
from mp_api.client import MPRester
import csv
from api_key import APIKEY
API_KEY = APIKEY

In [4]:
csv_file_path = 'new_MPIDS_forbandsearch.csv'
data = []

with open(csv_file_path, mode='r', encoding='utf-8') as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        data.append(row)

MPID_ids_list = [nested_id[0] for nested_id in data]

In [5]:
print(len(MPID_ids_list))

1985


In [6]:
materials = MPID_ids_list[]
bandstruct = []
failed_ids = []

with MPRester(API_KEY) as mpr:
    for mp_id in materials:
        try:
            band_structure = mpr.get_bandstructure_by_material_id(mp_id)
            bandstruct.append(band_structure)
        except Exception as e:
            print(f"Failed to retrieve band structure for {mp_id}: {e}")
            failed_ids.append(mp_id)

for id in failed_ids:
    materials.remove(id)

SyntaxError: invalid syntax (195131442.py, line 1)

In [ ]:
print(len(materials))

516


In [ ]:
mp_entries = []

with MPRester(API_KEY) as mpr:
    for mp_id in materials:
        material_entries = mpr.materials.search(material_ids=[mp_id])
        
        if material_entries:
            mp_entries.append(material_entries[0])
        
chemsys_list = [entry.chemsys for entry in mp_entries if hasattr(entry, 'chemsys')]
composition_list = [entry.composition_reduced for entry in mp_entries if hasattr(entry, 'composition_reduced')] 
materials_gga = [material + "-GGA" for material in materials]
materials_id = materials

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

...

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
print(mp_entries[0])

MPDataDoc<MaterialsDoc>
builder_meta=EmmetMeta(emmet_version='0.71.0', pymatgen_version='2023.10.4', pull_request=990, database_version='2023.11.1', build_date=datetime.datetime(2023, 10, 18, 21, 34, 37, 413000), license='BY-C'),
nsites=100,
elements=[Element O, Element Pr, Element Se],
nelements=3,
composition=Composition('Pr40 Se56 O4'),
composition_reduced=Composition('Pr10 Se14 O1'),
formula_pretty='Pr10Se14O',
formula_anonymous='AB10C14',
chemsys='O-Pr-Se',
volume=2642.9093320437887,
density=6.35968924042202,
density_atomic=26.429093320437886,
symmetry=SymmetryData(crystal_system=<CrystalSystem.tet: 'Tetragonal'>, symbol='I4_1/acd', number=142, point_group='4/mmm', symprec=0.1, version='2.0.2'),
material_id=MPID(mp-560835),
structure=Structure Summary
Lattice
    abc : 15.364236697313537 15.364236697313537 15.364236697313537
 angles : 117.81494910786272 117.81494910786272 93.82820940998708
 volume : 2642.9093320437887
      A : -7.934424 7.934424 10.495218
      B : 7.934424 -7.93

In [ ]:
print(len(mp_entries),len(bandstruct))

516 516


In [ ]:
chemsys_list = [entry.chemsys for entry in mp_entries if hasattr(entry, 'chemsys')]
composition_list = [entry.composition_reduced for entry in mp_entries if hasattr(entry, 'composition_reduced')] 
materials_gga = [material + "-GGA" for material in materials]
materials_id = materials

In [ ]:
bandgaps = []

for i in bandstruct:
    bandgap = i.get_band_gap()
    bandgapenergy = bandgap.get('energy')
    bandgaps.append(bandgapenergy)

electroneg = []

for i in composition_list:
    electroneg.append(i.average_electroneg)

NameError: name 'bandstruct' is not defined

In [ ]:
electroneg = []

for i in composition_list:
    electroneg.append(i.average_electroneg)

In [ ]:
cbm = []
vbm = []

for i,x in zip(bandgaps,electroneg):

    bandcorrection = (i + 0.85)/2
    cbm_pre = x + bandcorrection
    vbm_pre = x - bandcorrection
    cbm_minusvac = cbm_pre - 4.5
    vbm_minusvac = vbm_pre - 4.5

    cbm.append(cbm_minusvac)
    vbm.append(vbm_minusvac)

In [ ]:
band_edge = list(zip(cbm, vbm, materials))

In [ ]:
CO_reduction_met = [item for item in band_edge if item[0] >= 0.12]
CHOH_reduction_met = [item for item in band_edge if 0 <= item[0] < 0.12]
CH4_reduction_met = [item for item in band_edge if -0.17 <= item[0] < 0]

In [ ]:
print("CO:",len(CO_reduction_met),"||","CHOH:",len(CHOH_reduction_met),"||","CH4:",len(CH4_reduction_met))

CO: 10 || CHOH: 17 || CH4: 29


In [ ]:
print(len(failed_ids))

469


In [ ]:
#print("CO:",CO_reduction_met)
CO_mpids = [item[2] for item in CO_reduction_met]
print(CO_mpids)

['mp-1397055', 'mp-768936', 'mp-760803', 'mp-759856', 'mp-735822', 'mp-759097', 'mp-19077', 'mp-759764', 'mp-763903', 'mp-759867']


In [ ]:
#print("CHOH:",CHOH_reduction_met)
CHOH_mpids = [item[2] for item in CHOH_reduction_met]
print(CHOH_mpids)

['mp-554944', 'mp-771567', 'mp-781604', 'mp-10798', 'mp-776596', 'mp-851257', 'mp-760358', 'mp-760912', 'mp-767435', 'mp-766954', 'mp-723285', 'mp-779365', 'mp-754461', 'mp-760919', 'mp-560748', 'mp-760914', 'mp-645799']


In [ ]:
#print("CH4:",CH4_reduction_met)
CH4_mpids = [item[2] for item in CH4_reduction_met]
print(CH4_mpids)

['mp-763169', 'mp-23170', 'mp-562239', 'mp-230', 'mp-31069', 'mp-1008492', 'mp-23178', 'mp-25407', 'mp-19067', 'mp-759886', 'mp-777890', 'mp-25250', 'mp-777342', 'mp-778865', 'mp-722980', 'mp-1111368', 'mp-27475', 'mp-555549', 'mp-775103', 'mp-759307', 'mp-1190551', 'mp-777880', 'mp-624234', 'mp-616232', 'mp-504945', 'mp-759085', 'mp-778749', 'mp-849409', 'mp-764879']


In [ ]:
#cbms_CO = [item[0] for item in CO_reduction_met]
#vbms_CO = [item[1] for item in CO_reduction_met]
#mps_CO = [item[2] for item in CO_reduction_met]

#cbms_CHOH = [item[0] for item in CHOH_reduction_met]
#vbms_CHOH = [item[1] for item in CHOH_reduction_met]
#mps_CHOH = [item[2] for item in CHOH_reduction_met]

In [ ]:
'''
import matplotlib.pyplot as plt

water_oxidation = -1.23
co_reduction = 0.12

plt.figure(figsize=(6, 4))

for i in range(len(CO_reduction_met)):
    plt.hlines(cbms_above[i], i - 0.1, i + 0.1, colors='blue', label='CBM' if i == 0 else "")
    plt.hlines(vbms_above[i], i - 0.1, i + 0.1, colors='red', label='VBM' if i == 0 else "")

plt.axhline(y=water_oxidation, color='green', linestyle='--', label='Water Oxidation')
plt.axhline(y=co_reduction, color='green', linestyle='--', label='CO2 Redox')

# Adding labels and title
plt.ylabel('Energy (eV)')
plt.title('Band Gap Alignment: Above CO Reduction')
plt.legend()

# Show the plot
plt.show()

'''


'\nimport matplotlib.pyplot as plt\n\nwater_oxidation = -1.23\nco_reduction = 0.12\n\nplt.figure(figsize=(6, 4))\n\nfor i in range(len(CO_reduction_met)):\n    plt.hlines(cbms_above[i], i - 0.1, i + 0.1, colors=\'blue\', label=\'CBM\' if i == 0 else "")\n    plt.hlines(vbms_above[i], i - 0.1, i + 0.1, colors=\'red\', label=\'VBM\' if i == 0 else "")\n\nplt.axhline(y=water_oxidation, color=\'green\', linestyle=\'--\', label=\'Water Oxidation\')\nplt.axhline(y=co_reduction, color=\'green\', linestyle=\'--\', label=\'CO2 Redox\')\n\n# Adding labels and title\nplt.ylabel(\'Energy (eV)\')\nplt.title(\'Band Gap Alignment: Above CO Reduction\')\nplt.legend()\n\n# Show the plot\nplt.show()\n\n'

In [ ]:
'''
water_oxidation = -1.23
co_reduction = 0.12

plt.figure(figsize=(6, 4))

for i in range(len(CO_reduction_below)):
    plt.hlines(cbms_below[i], i - 0.1, i + 0.1, colors='blue', label='CBM' if i == 0 else "")
    plt.hlines(vbms_below[i], i - 0.1, i + 0.1, colors='red', label='VBM' if i == 0 else "")

plt.axhline(y=water_oxidation, color='green', linestyle='--', label='Water Oxidation')
plt.axhline(y=co_reduction, color='green', linestyle='--', label='CO2 Redox')

# Adding labels and title
plt.ylabel('Energy (eV)')
plt.title('Band Gap Alignment: Below CO Reduction')
plt.legend()

# Show the plot
plt.show()
'''

'\nwater_oxidation = -1.23\nco_reduction = 0.12\n\nplt.figure(figsize=(6, 4))\n\nfor i in range(len(CO_reduction_below)):\n    plt.hlines(cbms_below[i], i - 0.1, i + 0.1, colors=\'blue\', label=\'CBM\' if i == 0 else "")\n    plt.hlines(vbms_below[i], i - 0.1, i + 0.1, colors=\'red\', label=\'VBM\' if i == 0 else "")\n\nplt.axhline(y=water_oxidation, color=\'green\', linestyle=\'--\', label=\'Water Oxidation\')\nplt.axhline(y=co_reduction, color=\'green\', linestyle=\'--\', label=\'CO2 Redox\')\n\n# Adding labels and title\nplt.ylabel(\'Energy (eV)\')\nplt.title(\'Band Gap Alignment: Below CO Reduction\')\nplt.legend()\n\n# Show the plot\nplt.show()\n'